In [ ]:
!pip install transformers &> /dev/null

In [ ]:
from os.path import join
import glob
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from transformers import XLMTokenizer, RobertaModel
import tensorflow as tf
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as data_utils
import numpy as np
from typing import List
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import itertools
from tensorflow.keras import layers, models
import seaborn as sn

In [ ]:
tqdm.pandas()

## Mount Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Data loading

In [ ]:
data = pd.read_csv("/content/drive/My Drive/nlp/wiki_pages.csv", index_col=0)

In [ ]:
data.head()

,title,text,category
0,Mirosław_Miller,Mirosław Miller – Dyrektor Międzynarodowego La...,Chemia
1,Chimerokształtne,"Chimerokształtne, chimery, przerazy (Chimaerif...",Biologia
2,(2855)_Bastian,(2855) Bastian (1931 TB2) – planetoida z grupy...,Astronomia
3,Cryptocephalus_celtibericus,"""Cryptocephalus celtibericus"" – gatunek chrząs...",Biologia
4,Język_maszynowy,"Język maszynowy, kod maszynowy – zestaw rozkaz...",Matematyka


In [ ]:
data.shape

(75792, 3)

In [ ]:
data.dropna(inplace=True)

In [ ]:
# data = data.iloc[:25264]
# data = data.iloc[25264:50528]
data = data.iloc[50528:]

In [ ]:
data.shape

(24915, 3)

## Embedding

In [ ]:
tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

#### Model test

In [ ]:
encoded_input = tokenizer.encode(data.iloc[2]['text'], return_tensors='pt')
outputs = model(encoded_input)

In [ ]:
encoded_input.shape

torch.Size([1, 211])

In [ ]:
len(data.iloc[0]['text'].split())

358

In [ ]:
# output of each layer
outputs[0].shape

torch.Size([1, 211, 768])

In [ ]:
# output of the embeddings
len(outputs[1].resize(outputs[1].shape[1]))

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


768

#### Data embedding

In [ ]:
# class FrameDataset(data_utils.Dataset):
#   def __init__(
#       self,
#       dataFrame: pd.DataFrame,
#       x_label: str,
#       y_label: str
#   ):
#     self.df = dataFrame
#     self.x = list(self.df[x_label])
#     self.y = list(self.df[y_label])

#   def __getitem__(self, idx):
#     return self.x[idx], self.y[idx]

#   def __len__(self):
#     return len(self.y)


class Embedder(nn.Module):
  def __init__(self, tokenizer, embedder):
    super(Embedder, self).__init__()
    self.tokenizer = tokenizer
    self.embedder = model

  def forward(self, x):
    if type(x) == str and len(x) > 0:
      with torch.no_grad():
        x = self.tokenizer(x, padding=True, truncation=True, max_length=128, return_tensors='pt')
        x = self.embedder(**x)[1]
        x = x.numpy()
        x = x.reshape(x.shape[1])
    else:
      x = np.zeros(768)
    return x

In [ ]:
# data = FrameDataset(df, 'tweet', 'emotion')
# dataloader = data_utils.DataLoader(data, batch_size=100, shuffle=False)

embedder = Embedder(tokenizer, model)

In [ ]:
data['doc_vec'] = data.text.progress_apply(embedder)

100%|██████████| 24915/24915 [2:46:43<00:00,  2.49it/s]


In [ ]:
# embeddings = []
# for i, data in tqdm(enumerate(dataloader, 0)):

#     inputs, labels = data
#     outputs = embedder(inputs)
#     embeddings.append(outputs.numpy())

In [ ]:
# data.to_csv("/content/drive/My Drive/nlp/wiki_pages_vec_1.csv")
# data.to_csv("/content/drive/My Drive/nlp/wiki_pages_vec_2.csv")
data.to_csv("/content/drive/My Drive/nlp/wiki_pages_vec_3.csv")

In [ ]:
data.head()

,title,text,category,doc_vec
50771,LOFAR,Centralne skupisko anten w Holandii\nLOFAR (– ...,Astronomia,"[0.29673636, 0.16016921, -0.12867889, 0.303704..."
50772,Diagram_Latimera,Diagram Latimera – wykres stosowany w elektroc...,Fizyka,"[-0.15167649, -0.25634798, 0.23790883, 0.25928..."
50773,Stanisław_Michna,"Stanisław Władysław Michna (ur. 1907, zm. 2005...",Biologia,"[0.37255284, -0.06458719, 0.4710701, 0.4513134..."
50774,Prosopocoilus_astacoides_blanchardi,"""Prosopocoilus astacoides blanchardi"" – podgat...",Biologia,"[0.018108765, 0.05765905, 0.20286876, 0.511124..."
50775,Eksperyment_Fizeau,"e=""preserve"">Szkic aparatury do eksperymentu F...",Fizyka,"[0.27692065, -0.14168482, 0.282383, 0.22537696..."


In [ ]:
data.iloc[99]['doc_vec'].shape

(768,)